# 🌴 Asistente Turístico de Tenerife - RAG Demo

## 1. Configuración del Entorno
Cargamos las librerías necesarias y verificamos la API Key.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"✅ Key cargada: {api_key[:5]}...")
else:
    print("❌ Error de Key")

✅ Key cargada: sk-pr...


In [2]:
# 0. INSTALACIÓN DE DEPENDENCIAS
# Ejecuta esto PRIMERO. Si hay actualizaciones, Reinicia el Kernel antes de seguir.
%pip install -q -U openai-agents nest_asyncio openai langchain-openai python-dotenv
print("✅ Dependencias verificadas. Si hubo instalaciones nuevas, ve a Kernel > Restart.")

Note: you may need to restart the kernel to use updated packages.
✅ Dependencias verificadas. Si hubo instalaciones nuevas, ve a Kernel > Restart.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Validación de Conectividad con OpenAI
Probamos que la llave funcione y el modelo responda.

In [3]:
from langchain_openai import ChatOpenAI

# Inicializamos el modelo
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Prueba 'Holística'
try:
    response = llm.invoke("Di 'Hola Tenerife' si me escuchas.")
    print(f"🤖 Respuesta Modelo: {response.content}")
except Exception as e:
    print(f"❌ Error de Conexión: {e}")

🤖 Respuesta Modelo: Hola Tenerife. ¿En qué puedo ayudarte hoy?


## 3. Fase 2: Ingesta y Vector Store 📚
Subimos el PDF `TENERIFE.pdf` a OpenAI para crear un Vector Store gestionado.

In [4]:
import os
import sys
import openai
from openai import OpenAI
from pathlib import Path

# 1. DIAGNÓSTICO DE ENTORNO (CRÍTICO)
print(f"🐍 Python Executable: {sys.executable}")
if "llm-env" not in sys.executable:
    print("⚠️ ADVERTENCIA: No parece que estés usando el entorno virtual 'llm-env'. Verifica tu Kernel.")

print(f"📦 OpenAI Version: {openai.__version__}")

client = OpenAI()

pdf_path = Path("../data/raw/TENERIFE.pdf")

if not pdf_path.exists():
    print(f"❌ No encontrado: {pdf_path}")
else:
    print(f"✅ Archivo encontrado: {pdf_path}")

    # 2. ADAPTADOR UNIVERSAL DE VECTOR STORES
    try:
        vs_manager = None
        
        # Intento 1: Ruta estándar V2 Beta
        if hasattr(client, 'beta') and hasattr(client.beta, 'vector_stores'):
            vs_manager = client.beta.vector_stores
            print("✅ Conectado vía client.beta.vector_stores")
            
        # Intento 2: Ruta raíz (posible cambio futuro o versión diferente)
        elif hasattr(client, 'vector_stores'):
            vs_manager = client.vector_stores
            print("✅ Conectado vía client.vector_stores")

        if vs_manager:
            vector_store = vs_manager.create(name="Tenerife Guide Store")
            print(f"📦 Vector Store creado: {vector_store.id}")
            
            with open(pdf_path, "rb") as f:
                file_batch = vs_manager.file_batches.upload_and_poll(
                    vector_store_id=vector_store.id, files=[f]
                )
            print(f"📄 Estado de carga: {file_batch.status}")
            print(f"🔢 Archivos procesados: {file_batch.file_counts}")
        else:
            # FALLO TOTAL: Imprimir introspección para depurar
            print("❌ ERROR: No se encuentra 'vector_stores' en el cliente OpenAI.")
            print("🔍 Depuración de atributos disponibles:")
            print(f"   client dir: {[x for x in dir(client) if 'vector' in x or 'beta' in x]}")
            if hasattr(client, 'beta'):
                print(f"   client.beta dir: {dir(client.beta)}")
            else:
                print("   client.beta NO existe.")
            raise AttributeError("No se pudo acceder a Vector Stores con esta versión de librería/entorno.")

    except Exception as e:
        print(f"❌ Excepción: {e}")

🐍 Python Executable: f:\development\Development\Master IA\LLM-large-language-models-entrega-tarea-final\llm-env\Scripts\python.exe
📦 OpenAI Version: 2.15.0
✅ Archivo encontrado: ..\data\raw\TENERIFE.pdf
✅ Conectado vía client.vector_stores
📦 Vector Store creado: vs_69667113b8408191a1c6c814588cf19c
📄 Estado de carga: completed
🔢 Archivos procesados: FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


## 4. Fase 3: Configuración del Agente RAG 🤖
Configuramos el agente con capacidad de usar el Vector Store. 
**Nota**: Incluimos la instalación de librerías en el kernel para asegurar que el entorno de ejecución tenga todo lo necesario.

In [5]:
import nest_asyncio
nest_asyncio.apply()

# Corrección: La librería se importa como 'agents' aunque se instale como 'openai-agents'
from agents import Agent, Runner, FileSearchTool

# Recuperamos el ID del Vector Store de la fase anterior
# Fallback a un ID conocido si se ha reiniciado el kernel sin re-ejecutar la Fase 2
try:
    current_vs_id = vector_store.id
except NameError:
    current_vs_id = "vs_69661c161c5881919eadf77744958c70" # ID de ejemplo/previo
    print(f"⚠️ Usando ID recuperado/hardcodeado: {current_vs_id}")

# 1. Definir la herramienta de búsqueda de archivos
file_search_tool = FileSearchTool(vector_store_ids=[current_vs_id])

# 2. Instrucciones del Agente (System Prompt)
INSTRUCTIONS = """
Eres un experto guía turístico de Tenerife.
Usa la herramienta file_search para encontrar información en la guía PDF adjunta.
Si la información no está en el PDF, dilo claramente. No inventes respuestas.
Responde siempre en español, con un tono amable y entusiasta.
"""

# 3. Crear el Agente
agent = Agent(
    name="TenerifeGuide",
    model="gpt-4o-mini",
    instructions=INSTRUCTIONS,
    tools=[file_search_tool]
)

print(f"🤖 Agente '{agent.name}' configurado satisfactoriamente.")

🤖 Agente 'TenerifeGuide' configurado satisfactoriamente.


### Prueba Rápida del Agente
Lanzamos una pregunta de prueba para verificar que el RAG funciona.

In [6]:
async def demo_agente():
    pregunta = "¿Qué actividades puedo hacer en el Teide?"
    print(f"👤 Usuario: {pregunta}")
    
    # Ejecutamos el agente
    resultado = await Runner.run(agent, pregunta)
    print(f"🤖 Agente: {resultado.final_output}")

await demo_agente()

👤 Usuario: ¿Qué actividades puedo hacer en el Teide?


AttributeError: 'RunResult' object has no attribute 'final_response'